In [1]:
from smoothie.auth.AuthorizationHandler import AuthorizationHandler
from smoothie.data_objects.User import User
from smoothie.data_objects.Song import Song
from smoothie.data_objects.Playlist import Playlist
import aiohttp
import urllib
import getpass
import spotify

In [29]:
# Reimport any changes
import importlib
importlib.reload(spotify.playlist_apis)

<module 'spotify.playlist_apis' from '/home/jason/projects/smoothie/smoothie/spotify/playlist_apis.py'>

In [3]:
session = aiohttp.ClientSession()
auth_handler = AuthorizationHandler(session)

Since Spotify requires a login page with user interaction, we'll need to perform this first step in a browser page. Copy and paste the result of the next cell into a browser and approve access to your Spotify account.

In [4]:
print(f'https://accounts.spotify.com/authorize?client_id={auth_handler.spotify_client_id}&response_type=code&redirect_uri={urllib.parse.quote_plus(auth_handler._redirect_uri)}')

https://accounts.spotify.com/authorize?client_id=c6571c2bc3f444a18ad0308ab4b712f3&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com


Now, in the resulting `google.com` url, copy all the text after `code=`, and enter it when prompted in the next cell

In [8]:
code = getpass.getpass(prompt='Code from URL? ')
me = User(code=code)

Code from URL? ········


Get access/refresh tokens for use with future API calls

In [9]:
await auth_handler.request_access_token(me)

{'access_token': 'BQBC1rr_ygdFA_-y1a3kkye4ZNLxxTUAwTcNUYrgP7_8iQdZ1RTz560QDoaITS0g2XeBYhxr33uqkKBmyWffhSW3R670duLn4iHpHI6QLvDQqz2I3PSe_PR6-hDi3EzO4cmPdR5dlYkB1KdX_A',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQAhJMOl-1u7l7AI6UsUAOB6oB0KCoGlONvswSIbu7fC-nmvrxD86ps6moivbl_jy7GTj4YfRlHfJUNXii8adRF6qV-YL9J0uXsezfluSlONVTC8bd6UuIUhFlYTozY4EuE'}

In [30]:
await spotify.user_apis.get_profile(session, me)
playlist_list = await spotify.playlist_apis.get_current_users_playlists(session, me)

audio_features = []

for playlist in playlist_list:
    track_list = await spotify.playlist_apis.get_playlist(session, me, playlist['id'])
    for i in range(int(len(track_list) / 100 + 1)):
        id_list = [track['track']['id'] for track in track_list[100 * i: min(100*(i+1), len(track_list))]]
        if not id_list:
            print(id_list)
            print(len(track_list))
            print(i)
        _audio_features = await spotify.track_apis.get_audio_features_for_several_tracks(session, me, id_list)
        audio_features += _audio_features

print(audio_features)


{'tracks': {'items': [{'added_at': '2018-06-06T18:57:47Z', 'track': {'id': '5iUQMwxUPdJBFeGkePtM66'}}, {'added_at': '2018-06-06T18:58:54Z', 'track': {'id': '1pynf76l80VPFR8pEQRLOu'}}, {'added_at': '2018-06-06T19:00:20Z', 'track': {'id': '6JvfBzqZmSiEG5MjM7OcSY'}}, {'added_at': '2018-06-06T19:00:44Z', 'track': {'id': '1h2LHhmyAXi8tPNLi806JA'}}, {'added_at': '2018-06-06T19:00:49Z', 'track': {'id': '564oa00vY05d1uYnTEAAmE'}}, {'added_at': '2018-06-06T19:00:55Z', 'track': {'id': '6i4SkbWVHe0pEP9PrreQ7A'}}, {'added_at': '2018-06-06T19:01:00Z', 'track': {'id': '67pIYW0MEQWWkNvuXb70vC'}}, {'added_at': '2018-06-06T19:01:06Z', 'track': {'id': '6zWU7YALeEDMcPGhKKZJhV'}}, {'added_at': '2018-06-06T19:01:11Z', 'track': {'id': '3ml3WB66sqG4kXoPBX2POA'}}, {'added_at': '2018-06-06T19:01:15Z', 'track': {'id': '5HycJMEM8MyX4i0ErSeuC4'}}, {'added_at': '2018-06-06T19:02:08Z', 'track': {'id': '7ox2wvSzqTxQQf55Ggobbl'}}, {'added_at': '2018-06-06T19:02:13Z', 'track': {'id': '2p4ARx5nk3b9Jh07ivQR1c'}}, {'adde

KeyError: 'tracks'

In [ ]:
songs = [Song(audio_feature) for audio_feature in audio_features]
playlist = Playlist()
playlist.add_songs(songs)